# **Student Bayes Network**

## Modelado de Red Bayesiana 

In [1]:
"""
Definición de relaciones en la Red Bayesiana
"""

def BayesNetwork():
    D = {'cpd': {0: 0.6, 1: 0.4},
         'children': None}
    
    I = {'cpd': {0: 0.7, 1: 0.3},
         'children': None}
    
    S = {'cpd': {0: {0:0.95, 1:0.2},
                 1: {0:0.05, 1:0.8}},
         'children': {'Intelligence':0}}
    
    G = {'cpd':{ 1: {(0,0):0.3, (0,1):0.05, (1,0):0.9 , (1,1):0.5},
                 2: {(0,0):0.4, (0,1):0.25, (1,0):0.08, (1,1):0.3},
                 3: {(0,0):0.3, (0,1):0.7 , (1,0):0.02, (1,1):0.2}},
         'children': {'Intelligence':0, 'Difficulty':1}}
    
    L = {'cpd':{0: {1:0.1 , 2:0.4, 3:0.99},
                1: {1:0.9 , 2:0.6, 3:0.01}},
         'children': {'Grade':0}}

    nodesLinked = {'Difficulty': D, 'Intelligence': I, 'Grade': G, 'SAT': S, 'Letter': L}

    return nodesLinked

<br/>

## Entramiento de Rede Bayesiana

In [2]:
"""
Operaciones sobre factores
"""
def operateFactores(cpds, children, BayNet, deep, res):
    sump = 0.0
    res_prob  = {}
    if children:
        rev_chdic = dict(zip(children.values(), children.keys()))
    for key_state, children_states in cpds.items():
        if isinstance(children_states,(int,float)):
            res_prob[key_state] = children_states
            sump               += children_states
        else:
            sum_state = 0
            for children_state,p in children_states.items():
                prob_state = p
                if isinstance(children_state, (int,float)):
                    if rev_chdic[0] in res:
                        prob_state *= res[rev_chdic[0]][children_state]

                else:
                    for i, child_state in enumerate(children_state):
                        if rev_chdic[i] in res:
                            prob_state *= res[rev_chdic[i]][child_state]
                        else:
                            prob_state *=  BayNet[rev_chdic[i]]['cpd'][child_state]
                sum_state      += prob_state
            res_prob[key_state] = sum_state
            sump               += sum_state

    for key, val in res_prob.items():
        res_prob[key] = float('%.4f'%(val/sump))
    return res_prob

In [3]:
"""
Propagación de probabilidades en la red 
"""
def setProbability(res, key, BayNet, deep):
    if BayNet[key]['children']:
        deep +=1
        for child in BayNet[key]['children']:
            setProbability(res, child, BayNet, deep) 
        res[key] = operateFactores(BayNet[key]['cpd'], BayNet[key]['children'], BayNet, 0, res)
    else:
        if key not in res:
            res[key] = operateFactores(BayNet[key]['cpd'], None, BayNet, deep, res)

In [4]:
"""
Propagación de evidencias en la red
"""
def setEvidence(evidence, res, BayNet):
    ev_prob = {}
    for ev_state in BayNet[evidence[0]]['cpd']:
        if ev_state == evidence[1]:
            ev_prob[ev_state] = 1.0
        else:
            ev_prob[ev_state] = 0.0
    res[evidence[0]] = ev_prob

In [5]:
"""
Propagación de evidencias a los hijos de los nodos en la red
"""
def setEvidenceToChildren(res, children, evidenceParent, BayNet):
    if children:
        for child in children:
            res[child] = operateFactores(BayNet[child]['cpd'], BayNet[child]['children'], BayNet, 0)
            for s, p in res[child].items():
                res[child][s] = evidenceParent[s]*res[child][s]

            res[child] = operateFactores(res[child], None, BayNet, 0, res)
            setEvidenceToChildren(res, BayNet[child]['children'], res[child], BayNet)

In [6]:
"""
Entrenamiento de la red basado en el cáculo de probalidades
"""

def trainBayesNetwork(evidences=None, verbose=True):
    BayNet = BayesNetwork()
        
    res = {}

    if evidences:
        for evidence in evidences:
            if evidence[0] in BayNet:
                setEvidence(evidence, res, BayNet)
                setEvidenceToChildren(res, BayNet[evidence[0]]['children'], BayNet[evidence[0]]['cpd'][evidence[1]], BayNet)                        
            else:
                print("Evidence's name not found!\n")

    for key, node in BayNet.items():
        if verbose:
            print('key :', key)
        if key not in res:
            setProbability(res, key, BayNet,0) 
    return res

In [7]:
""""
Inicialización del entrenamiento de la red
"""
results = trainBayesNetwork()

key : Difficulty
key : Intelligence
key : Grade
key : SAT
key : Letter


<br/>

## Visualización de resultados en la Red Bayesiana

In [8]:
"""
Visualización de Resultados
"""
def printBayesNetwork(results):
    for key, val in results.items():
        print('{:10}\t\t: {}'.format(key, val))

In [9]:
printBayesNetwork(results)

Difficulty		: {0: 0.6, 1: 0.4}
Intelligence		: {0: 0.7, 1: 0.3}
Grade     		: {1: 0.362, 2: 0.2884, 3: 0.3496}
SAT       		: {0: 0.725, 1: 0.275}
Letter    		: {0: 0.4977, 1: 0.5023}


<br/>

## Comparación con SAMIAM

### Carga de Datos en SAMIAM

<img src='images/bn_src.PNG'>

### Compilación de Red

In [10]:
%%html
<img src='images/bn_out.PNG'>

De los datos mostrados se puede apreciar que se obtienen los mismos resultados usando **SAMIAM** y el Algoritmo de **Eliminación de Variables**.

<br/>

## Comparación con SAMIAM condicional

<img src='images/bn_cond.PNG'>

In [11]:
results = trainBayesNetwork(evidences = [('Difficulty',0)], verbose=False)

In [12]:
printBayesNetwork(results)

Difficulty		: {0: 1.0, 1: 0.0}
Intelligence		: {0: 0.7, 1: 0.3}
Grade     		: {1: 0.48, 2: 0.304, 3: 0.216}
SAT       		: {0: 0.725, 1: 0.275}
Letter    		: {0: 0.3834, 1: 0.6166}


Nuevamente se observa la equivalencia de resultados entre usar **SAMIAM** y el Algoritmo de **Eliminación de Variables**.